<a href="https://colab.research.google.com/github/aivscovid19/data_pipeline/blob/yeshwanth/URL_Builders/Biorxiv_URLBuilder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing Centaurminer chrome driver
!pip install centaurMiner==0.0.8
# Colab only: This is normally done automatically
!apt-get update # update ubuntu to correctly run apt-install
!apt install chromium-chromedriver # Installs to '/usr/lib/chromium-browser/chromedriver'
import time
import pandas_gbq
import pandas as pd
from datetime import datetime

     |████████████████████████████████| 911kB 3.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [36.0 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates In

In [2]:
import centaurminer as mining

In [ ]:
from google.colab import auth      # Google Authentication
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
project_id='for-yr'
table_id=f'Medical_Dataset.biorxiv_urll'
schema = [
    {'name': 'article_url', 'type': 'STRING',   'mode': 'REQUIRED'},
    {'name': 'catalog_url', 'type': 'STRING',   'mode': 'REQUIRED'},
    {'name': 'is_pdf',      'type': 'INT64',    'mode': 'REQUIRED'},
    {'name': 'language',    'type': 'STRING'                      },
    {'name': 'status',      'type': 'STRING',   'mode': 'REQUIRED'},
    {'name': 'timestamp',   'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'worker_id',   'type': 'STRING'                      },
    {'name': 'meta_info',   'type': 'STRING'                      }
]

In [4]:
class URL_builder():
  """ 
        Function to get the urls from the site with a search_word. 
  
        Parameters:
          search_word: word to be searched for urls. 
          pages: Number of pages from which the urls need to extracted.
  """
  def __init__(self,search_word,pages):
    """ 
        Function to get the urls from the site with a search_word. 
  
        Parameters:
          search_word: word to be searched for urls. 
          pages: Number of pages from which the urls need to extracted.
    """
    
    max_pages=pages+1
    url_search=f"https://www.biorxiv.org/search/{search_word}%20numresults%3A75%20sort%3Arelevance-rank?".format(search_word)
    article_set = set()
    driver_path = '/usr/lib/chromium-browser/chromedriver'
    self.url1=[]
    for page_num in range(1,max_pages):
      print(f"Scraping from page {page_num}...", flush=True)
      url=url_search+"page_num="+str(page_num) 
      articles = mining.Element("css_selector","a.highwire-cite-linked-title").get_attribute('href')
      urls = mining.CollectURLs(url, articles, driver_path=driver_path)
      [self.url1.append(i) for i in urls]
      self.url_schema=pd.DataFrame({'article_url': self.url1})
      self.url_schema['catalog_url']=url
      [article_set.add(i) for i in urls]
    self.url_schema['is_pdf']='0'
    self.url_schema['language']='en'
    self.url_schema['status']='not mined'
    self.url_schema['timestamp']= datetime.utcnow()
    self.url_schema['worker_id']=''
    self.url_schema['meta_info']=''
    print("Total of " + str(len(article_set)) + " articles")
  
  def bigquery(self,project_id,table_id,schema):#,project_id,table_id,schema):

    """ 
        Function to push the urls to the bigquery. 
  
        Parameters:
          project_id: project_id in bigquery. 
          table_id: table name.
          schema: schema to be followed.
    """
    # Append the new urls to the existing table
    pandas_gbq.to_gbq(self.new_urls, table_id, project_id=project_id, if_exists='append', table_schema=schema)
    return self.url_schema

In [5]:
urls=URL_builder('coronavirus',1)
urls.bigquery(project_id,table_id,schema)

Scraping from page 1...
Headless: True
driver path: /usr/lib/chromium-browser/chromedriver
Total of 75 articles


,article_url,catalog_url,is_pdf,language,status,timestamp,worker_id,meta_info
0,https://www.biorxiv.org/content/10.1101/2020.0...,https://www.biorxiv.org/search/coronavirus%20n...,0,en,not mined,2020-09-30 15:13:50.695861,,
1,https://www.biorxiv.org/content/10.1101/2020.0...,https://www.biorxiv.org/search/coronavirus%20n...,0,en,not mined,2020-09-30 15:13:50.695861,,
2,https://www.biorxiv.org/content/10.1101/2020.0...,https://www.biorxiv.org/search/coronavirus%20n...,0,en,not mined,2020-09-30 15:13:50.695861,,
3,https://www.biorxiv.org/content/10.1101/2020.0...,https://www.biorxiv.org/search/coronavirus%20n...,0,en,not mined,2020-09-30 15:13:50.695861,,
4,https://www.biorxiv.org/content/10.1101/2020.0...,https://www.biorxiv.org/search/coronavirus%20n...,0,en,not mined,2020-09-30 15:13:50.695861,,
...,...,...,...,...,...,...,...,...
70,https://www.biorxiv.org/content/10.1101/2020.0...,https://www.biorxiv.org/search/coronavirus%20n...,0,en,not mined,2020-09-30 15:13:50.695861,,
71,https://www.biorxiv.org/content/10.1101/2020.0...,https://www.biorxiv.org/search/coronavirus%20n...,0,en,not mined,2020-09-30 15:13:50.695861,,
72,https://www.biorxiv.org/content/10.1101/2020.0...,https://www.biorxiv.org/search/coronavirus%20n...,0,en,not mined,2020-09-30 15:13:50.695861,,
73,https://www.biorxiv.org/content/10.1101/2020.0...,https://www.biorxiv.org/search/coronavirus%20n...,0,en,not mined,2020-09-30 15:13:50.695861,,
